In [1]:
#@title 🎯 Fukuoka MQ Challenge Solver - Quaternion Dynamics + PySAT
#@markdown ## Complete solver for MQ Challenge (Type I, II, III, IV, V, VI)
#@markdown ## Converts MQ problems to SAT and solves with hybrid Quaternion + CDCL

!pip install python-sat requests beautifulsoup4 -q

import numpy as np
import lzma
import gzip
import math
import time
import requests
import os
import re
from io import BytesIO
from google.colab import files
from pysat.solvers import Solver as PySolver
from collections import defaultdict

# ============================================================
# STABLE QUATERNION CORE
# ============================================================
class Quaternion:
    """Stable quaternion implementation for SAT heuristics"""
    __slots__ = ['q']

    def __init__(self, w=0.0, x=0.0, y=0.0, z=0.0):
        self.q = np.array([w, x, y, z], dtype=np.float64)
        self._stabilize()

    def _stabilize(self):
        self.q = np.clip(self.q, -1e15, 1e15)
        self.q = np.nan_to_num(self.q, nan=0.0, posinf=1e10, neginf=-1e10)
        max_val = np.max(np.abs(self.q))
        if max_val > 1e8:
            self.q /= max_val

    @property
    def w(self): return self.q[0]
    @property
    def x(self): return self.q[1]
    @property
    def y(self): return self.q[2]
    @property
    def z(self): return self.q[3]

    def __add__(self, other):
        r = Quaternion()
        r.q = self.q + other.q
        r._stabilize()
        return r

    def __mul__(self, other):
        if isinstance(other, (int, float)):
            r = Quaternion()
            r.q = self.q * other
            r._stabilize()
            return r

        a, b, c, d = self.q
        e, f, g, h = other.q

        r = Quaternion()
        r.q = np.array([
            a*e - b*f - c*g - d*h,
            a*f + b*e + c*h - d*g,
            a*g - b*h + c*e + d*f,
            a*h + b*g - c*f + d*e
        ], dtype=np.float64)
        r._stabilize()
        return r

    def norm(self):
        return np.sqrt(np.sum(self.q ** 2))

    def to_matrix(self):
        w, x, y, z = self.q
        return np.array([
            [w, -x, -y, -z],
            [x,  w, -z,  y],
            [y,  z,  w, -x],
            [z, -y,  x,  w]
        ], dtype=np.float64)


# ============================================================
# MQ PROBLEM PARSER (Fukuoka Format)
# ============================================================
class MQProblem:
    """
    Parser for Fukuoka MQ Challenge format
    Supports all challenge types: I, II, III, IV, V, VI
    """

    def __init__(self):
        self.n = 0  # Number of variables
        self.m = 0  # Number of equations
        self.field = 2  # GF(2) by default
        self.equations = []  # List of equations
        self.seed = None
        self.challenge_type = None

    def parse_challenge_file(self, content, filename=""):
        """Parse MQ Challenge format file"""
        lines = content.strip().split('\n')

        self.equations = []
        current_eq = []

        parsing_equations = False

        for line in lines:
            line = line.strip()

            # Skip empty lines and comments
            if not line or line.startswith('#') or line.startswith('c'):
                continue

            # Parse header information
            if line.startswith('Galois Field'):
                match = re.search(r'GF\((\d+)\)', line)
                if match:
                    self.field = int(match.group(1))
                continue

            if line.startswith('Number of variables'):
                match = re.search(r':\s*(\d+)', line)
                if match:
                    self.n = int(match.group(1))
                continue

            if line.startswith('Number of polynomials') or line.startswith('Number of equations'):
                match = re.search(r':\s*(\d+)', line)
                if match:
                    self.m = int(match.group(1))
                continue

            if line.startswith('Seed'):
                match = re.search(r':\s*(\d+)', line)
                if match:
                    self.seed = int(match.group(1))
                continue

            # Detect challenge type from filename
            if 'typeI' in filename.lower() or 'type1' in filename.lower():
                self.challenge_type = 'I'
            elif 'typeII' in filename.lower() or 'type2' in filename.lower():
                self.challenge_type = 'II'
            elif 'typeIII' in filename.lower() or 'type3' in filename.lower():
                self.challenge_type = 'III'
            elif 'typeIV' in filename.lower() or 'type4' in filename.lower():
                self.challenge_type = 'IV'
            elif 'typeV' in filename.lower() or 'type5' in filename.lower():
                self.challenge_type = 'V'
            elif 'typeVI' in filename.lower() or 'type6' in filename.lower():
                self.challenge_type = 'VI'

            # Parse equation lines (semicolon marks end of equation)
            if ';' in line or '+' in line or '*' in line or line.replace(' ','').replace('x','').replace('0','').replace('1','').replace('_','').isdigit():
                parsing_equations = True

            if parsing_equations:
                # Clean and accumulate equation parts
                current_eq.append(line.replace(';', ''))

                if ';' in line:
                    # End of equation
                    eq_str = ' '.join(current_eq)
                    parsed_eq = self._parse_equation(eq_str)
                    if parsed_eq:
                        self.equations.append(parsed_eq)
                    current_eq = []

        # Try alternative parsing if no equations found
        if not self.equations:
            self._parse_alternative_format(content)

        print(f"  Parsed MQ Problem:")
        print(f"    Field: GF({self.field})")
        print(f"    Variables (n): {self.n}")
        print(f"    Equations (m): {self.m}")
        print(f"    Parsed equations: {len(self.equations)}")
        if self.challenge_type:
            print(f"    Challenge Type: {self.challenge_type}")

        return len(self.equations) > 0

    def _parse_equation(self, eq_str):
        """
        Parse a single equation string
        Returns: dict with 'quadratic', 'linear', 'constant' terms
        """
        equation = {
            'quadratic': [],  # List of (i, j) pairs for x_i * x_j
            'linear': [],     # List of i for x_i
            'constant': 0     # 0 or 1
        }

        # Clean the equation string
        eq_str = eq_str.replace(' ', '').replace('\t', '')

        # Split by +
        terms = eq_str.split('+')

        for term in terms:
            term = term.strip()
            if not term:
                continue

            # Check for constant term
            if term == '1':
                equation['constant'] = 1
                continue
            if term == '0':
                continue

            # Check for quadratic term (x_i * x_j)
            if '*' in term:
                vars_match = re.findall(r'x_?(\d+)', term)
                if len(vars_match) >= 2:
                    i, j = int(vars_match[0]), int(vars_match[1])
                    equation['quadratic'].append((min(i,j), max(i,j)))
                elif len(vars_match) == 1:
                    # x_i * x_i = x_i in GF(2)
                    i = int(vars_match[0])
                    equation['linear'].append(i)
            else:
                # Linear term
                var_match = re.search(r'x_?(\d+)', term)
                if var_match:
                    i = int(var_match.group(1))
                    equation['linear'].append(i)

        return equation if (equation['quadratic'] or equation['linear'] or equation['constant']) else None

    def _parse_alternative_format(self, content):
        """Parse coefficient matrix format"""
        lines = content.strip().split('\n')

        # Try to detect coefficient format
        coef_lines = []
        for line in lines:
            line = line.strip()
            if line and not line.startswith('#') and not line.startswith('c'):
                # Check if line contains only 0s and 1s (coefficient line)
                cleaned = line.replace(' ', '').replace('\t', '')
                if all(c in '01' for c in cleaned) and len(cleaned) > 0:
                    coef_lines.append([int(c) for c in cleaned])

        if coef_lines:
            # Each line is coefficients for one equation
            for coeffs in coef_lines:
                equation = {
                    'quadratic': [],
                    'linear': [],
                    'constant': 0
                }

                # Determine structure based on coefficient count
                # For n variables: n*(n+1)/2 quadratic + n linear + 1 constant
                idx = 0

                # Quadratic terms
                for i in range(1, self.n + 1):
                    for j in range(i, self.n + 1):
                        if idx < len(coeffs) and coeffs[idx] == 1:
                            if i == j:
                                equation['linear'].append(i)  # x_i^2 = x_i in GF(2)
                            else:
                                equation['quadratic'].append((i, j))
                        idx += 1

                # Linear terms
                for i in range(1, self.n + 1):
                    if idx < len(coeffs) and coeffs[idx] == 1:
                        if i not in equation['linear']:
                            equation['linear'].append(i)
                    idx += 1

                # Constant term
                if idx < len(coeffs):
                    equation['constant'] = coeffs[idx]

                self.equations.append(equation)


# ============================================================
# MQ TO SAT CONVERTER
# ============================================================
class MQtoSATConverter:
    """
    Convert MQ problem over GF(2) to SAT (CNF)
    Uses algebraic normal form to CNF conversion
    """

    def __init__(self, mq_problem):
        self.mq = mq_problem
        self.num_vars = 0
        self.clauses = []
        self.var_map = {}  # Maps (type, indices) to SAT variable
        self.reverse_map = {}  # Maps SAT variable to description

    def convert(self):
        """Convert MQ problem to CNF clauses"""
        print("\n  Converting MQ to SAT...")

        # Original variables: x_1, ..., x_n
        for i in range(1, self.mq.n + 1):
            self._get_var('x', i)

        # Convert each equation
        for eq_idx, eq in enumerate(self.mq.equations):
            self._convert_equation(eq, eq_idx)

        print(f"    SAT Variables: {self.num_vars}")
        print(f"    SAT Clauses: {len(self.clauses)}")

        return self.clauses, self.num_vars, self.var_map, self.reverse_map

    def _get_var(self, var_type, *indices):
        """Get or create a SAT variable"""
        key = (var_type,) + indices
        if key not in self.var_map:
            self.num_vars += 1
            self.var_map[key] = self.num_vars
            self.reverse_map[self.num_vars] = key
        return self.var_map[key]

    def _convert_equation(self, eq, eq_idx):
        """
        Convert one equation to CNF clauses

        For GF(2): sum of terms ≡ 0 (mod 2)
        We use auxiliary variables for XOR constraints
        """
        terms = []

        # Collect all terms in the equation
        for (i, j) in eq['quadratic']:
            # Create auxiliary variable for x_i AND x_j
            and_var = self._get_var('and', i, j)
            xi = self._get_var('x', i)
            xj = self._get_var('x', j)

            # Encode: and_var ↔ (xi ∧ xj)
            # and_var → xi: (-and_var ∨ xi)
            # and_var → xj: (-and_var ∨ xj)
            # xi ∧ xj → and_var: (-xi ∨ -xj ∨ and_var)
            self.clauses.append((-and_var, xi))
            self.clauses.append((-and_var, xj))
            self.clauses.append((-xi, -xj, and_var))

            terms.append(and_var)

        for i in eq['linear']:
            xi = self._get_var('x', i)
            terms.append(xi)

        if eq['constant'] == 1:
            # Add a "True" constant (represented as auxiliary var set to True)
            const_var = self._get_var('const', eq_idx)
            self.clauses.append((const_var,))  # Force True
            terms.append(const_var)

        # Now encode: XOR of all terms = 0 (i.e., even number of True values)
        # This is the key constraint for GF(2) equations

        if len(terms) == 0:
            # Empty equation, always satisfied
            return
        elif len(terms) == 1:
            # Single term must be False
            self.clauses.append((-terms[0],))
        elif len(terms) == 2:
            # XOR(a, b) = 0 means a ↔ b
            # Encode: (a ∨ -b) ∧ (-a ∨ b)
            a, b = terms
            self.clauses.append((a, -b))
            self.clauses.append((-a, b))
        else:
            # For longer XOR, use chain of auxiliary variables
            self._encode_xor_chain(terms, eq_idx)

    def _encode_xor_chain(self, terms, eq_idx):
        """Encode XOR of multiple terms using auxiliary variables"""
        # XOR(t1, t2, ..., tn) = 0
        # Chain: aux1 = t1 XOR t2, aux2 = aux1 XOR t3, ..., final must be 0

        if len(terms) <= 1:
            if terms:
                self.clauses.append((-terms[0],))
            return

        current = terms[0]

        for i in range(1, len(terms)):
            next_term = terms[i]

            if i == len(terms) - 1:
                # Last XOR: result must be False
                # XOR(current, next_term) = 0 means current ↔ next_term
                self.clauses.append((current, -next_term))
                self.clauses.append((-current, next_term))
            else:
                # Intermediate XOR: create auxiliary variable
                aux = self._get_var('xor', eq_idx, i)

                # aux ↔ XOR(current, next_term)
                # aux ↔ (current ⊕ next_term)
                # Equivalent to: aux ↔ ((current ∨ next_term) ∧ ¬(current ∧ next_term))
                # CNF encoding for XOR:
                # (-aux ∨ -current ∨ -next_term)
                # (-aux ∨ current ∨ next_term)
                # (aux ∨ -current ∨ next_term)
                # (aux ∨ current ∨ -next_term)

                self.clauses.append((-aux, -current, -next_term))
                self.clauses.append((-aux, current, next_term))
                self.clauses.append((aux, -current, next_term))
                self.clauses.append((aux, current, -next_term))

                current = aux


# ============================================================
# HYBRID SAT SOLVER WITH QUATERNION HEURISTIC
# ============================================================
class SATSolverQuaternion:
    """Hybrid SAT Solver: Quaternion Heuristic + PySAT CDCL"""

    def __init__(self):
        self.num_vars = 0
        self.clauses = []
        self.var_pos = None
        self.var_neg = None
        self.var_clauses = None
        self.implications = None

    def load_clauses(self, clauses, num_vars):
        """Load clauses directly (from MQ conversion)"""
        self.clauses = [tuple(c) for c in clauses]
        self.num_vars = num_vars
        self._build_structures()
        return True

    def parse_cnf_content(self, content, filename=""):
        """Parse DIMACS CNF format"""
        self.clauses = []
        max_var = 0

        for line in content.split('\n'):
            line = line.strip()
            if not line or line[0] in 'cp%':
                if line.startswith('p cnf'):
                    parts = line.split()
                    if len(parts) >= 4:
                        self.num_vars = int(parts[2])
                continue

            clause = []
            for tok in line.split():
                try:
                    lit = int(tok)
                    if lit == 0:
                        if clause:
                            self.clauses.append(tuple(clause))
                            clause = []
                    else:
                        clause.append(lit)
                        max_var = max(max_var, abs(lit))
                except:
                    pass
            if clause:
                self.clauses.append(tuple(clause))

        self.num_vars = max(self.num_vars, max_var)
        self._build_structures()
        return len(self.clauses) > 0

    def _build_structures(self):
        """Build variable occurrence structures"""
        n = self.num_vars
        self.var_pos = np.zeros(n + 1, dtype=np.float64)
        self.var_neg = np.zeros(n + 1, dtype=np.float64)
        self.var_clauses = [[] for _ in range(n + 1)]
        self.implications = {i: set() for i in range(-n, n + 1) if i != 0}

        for idx, clause in enumerate(self.clauses):
            clause_len = len(clause)
            weight = 1.0 / clause_len

            for lit in clause:
                var = abs(lit)
                if var <= n:
                    if lit > 0:
                        self.var_pos[var] += weight
                    else:
                        self.var_neg[var] += weight
                    self.var_clauses[var].append(idx)

            if clause_len == 2:
                a, b = clause
                if abs(a) <= n and abs(b) <= n:
                    self.implications[-a].add(b)
                    self.implications[-b].add(a)

    def count_satisfied(self, assignments):
        """Count satisfied clauses"""
        count = 0
        for clause in self.clauses:
            for lit in clause:
                var = abs(lit)
                val = assignments.get(var, True)
                if (lit > 0 and val) or (lit < 0 and not val):
                    count += 1
                    break
        return count

    def _heuristic_quaternion(self):
        """Quaternion-based heuristic for initial variable assignment"""
        n_vars = self.num_vars
        n_clauses = len(self.clauses)

        assignments = {}
        forced = set()

        # Unit propagation
        for clause in self.clauses:
            if len(clause) == 1:
                lit = clause[0]
                var = abs(lit)
                assignments[var] = (lit > 0)
                forced.add(var)

        # Implication propagation
        changed = True
        passes = 0
        max_passes = int(np.log2(n_vars + 1)) + 3

        while changed and passes < max_passes:
            changed = False
            passes += 1

            for var in list(assignments.keys()):
                val = assignments[var]
                lit = var if val else -var

                if lit in self.implications:
                    for impl in self.implications[lit]:
                        impl_var = abs(impl)
                        impl_val = (impl > 0)

                        if impl_var not in assignments:
                            assignments[impl_var] = impl_val
                            forced.add(impl_var)
                            changed = True

        # Quaternion dynamics for remaining variables
        remaining_vars = [v for v in range(1, n_vars + 1) if v not in assignments]

        q_global = Quaternion(0, 0, 0, 0)

        for var in remaining_vars:
            if var > len(self.var_pos) - 1:
                assignments[var] = True
                continue

            pos = self.var_pos[var]
            neg = self.var_neg[var]
            total = pos + neg

            if total < 1e-10:
                assignments[var] = True
                continue

            A = var
            B = 1 if pos >= neg else -1
            C = len(self.var_clauses[var]) if var < len(self.var_clauses) else 0

            n = A - B + C
            if n == 0:
                n = 1

            a = (pos - neg) / (total + 1)
            b = total / (n_vars + 1)

            if b > 0:
                b_power = np.exp(np.log(b + 1) / max(abs(n), 1))
            else:
                b_power = 1.0

            q = Quaternion(
                w = a + b_power,
                x = B * pos / (total + 1),
                y = neg / (total + 1),
                z = np.log(abs(n) + 1) / np.log(n_vars + 2)
            )

            q_global = q_global + q

        n_coef = n_clauses / (n_vars + 1) + 1
        norm_g = q_global.norm()

        factor = np.log(norm_g + 1) / (n_coef + 1) if norm_g > 1e-10 else 1.0

        derivative = Quaternion(
            w = q_global.w * factor + q_global.x * 0.3,
            x = q_global.x * factor + q_global.y * 0.3,
            y = q_global.y * factor + q_global.z * 0.3,
            z = q_global.z * factor + q_global.w * 0.3
        )

        M = derivative.to_matrix()
        eigenvalues = np.linalg.eigvals(M)
        eigen_signs = np.sign(np.real(eigenvalues))
        matrix_values = M.flatten()

        for var in remaining_vars:
            if var in assignments:
                continue

            if var > len(self.var_pos) - 1:
                assignments[var] = True
                continue

            pos = self.var_pos[var]
            neg = self.var_neg[var]
            total = pos + neg

            idx = (var - 1) % 16
            val_matrix = matrix_values[idx]

            eigen_idx = (var - 1) % 4
            val_eigen = eigen_signs[eigen_idx]

            if total > 0:
                polarity = (pos - neg) / total
            else:
                polarity = 0

            clause_score = 0
            if var < len(self.var_clauses):
                for cl_idx in self.var_clauses[var]:
                    clause = self.clauses[cl_idx]
                    for lit in clause:
                        if abs(lit) == var:
                            weight = 1.0 / len(clause)
                            if lit > 0:
                                clause_score += weight
                            else:
                                clause_score -= weight
                            break

            score = (
                0.20 * np.sign(val_matrix) +
                0.15 * val_eigen +
                0.35 * np.sign(polarity) +
                0.30 * np.sign(clause_score)
            )

            assignments[var] = (score >= 0)

        # Fill any remaining variables
        for v in range(1, n_vars + 1):
            if v not in assignments:
                assignments[v] = True

        return assignments

    def _pysat_solve(self):
        """Use PySAT for exact SAT solving"""
        solver = PySolver(name='g3')
        for clause in self.clauses:
            solver.add_clause(list(clause))

        sat = solver.solve()
        if not sat:
            solver.delete()
            return "UNSAT", None

        model = solver.get_model()
        solver.delete()

        assignments = {}
        for lit in model:
            var = abs(lit)
            assignments[var] = (lit > 0)

        return "SAT", assignments

    def solve(self, timeout=None):
        """Main solving method"""
        n_vars = self.num_vars
        n_clauses = len(self.clauses)

        if n_clauses == 0:
            return "SAT", {i: True for i in range(1, n_vars + 1)}, 0, None

        print("  [1] Quaternionic approximation O(log n)...")
        heur_assign = self._heuristic_quaternion()
        sat_heur = self.count_satisfied(heur_assign)
        pct_heur = sat_heur * 100.0 / n_clauses
        print(f"      Heuristic: {sat_heur}/{n_clauses} ({pct_heur:.2f}%)")

        if sat_heur == n_clauses:
            return "SAT", heur_assign, np.log(n_vars + n_clauses + 1), heur_assign

        print("  [2] Exact decision with PySAT (CDCL)...")
        result, exact_assign = self._pysat_solve()

        return result, heur_assign, np.log(n_vars + n_clauses + 1), exact_assign


# ============================================================
# FUKUOKA MQ CHALLENGE MANAGER
# ============================================================
class FukuokaMQChallenge:
    """
    Complete Fukuoka MQ Challenge solver
    Downloads, parses, converts to SAT, and solves challenges
    """

    # Challenge URLs and parameters
    CHALLENGE_INFO = {
        'TypeI': {
            'description': 'Random dense MQ systems',
            'url_base': 'https://www.mqchallenge.org/challenges/typeI/',
            'sizes': [(37, 74), (39, 78), (41, 82), (43, 86), (45, 90)],
        },
        'TypeII': {
            'description': 'Random sparse MQ systems',
            'url_base': 'https://www.mqchallenge.org/challenges/typeII/',
            'sizes': [(55, 110), (60, 120), (65, 130), (70, 140)],
        },
        'TypeIII': {
            'description': 'MQ with special structure (Sflash-like)',
            'url_base': 'https://www.mqchallenge.org/challenges/typeIII/',
            'sizes': [(47, 47), (49, 49), (51, 51), (53, 53)],
        },
        'TypeIV': {
            'description': 'Over-determined MQ systems',
            'url_base': 'https://www.mqchallenge.org/challenges/typeIV/',
            'sizes': [(30, 90), (32, 96), (34, 102), (36, 108)],
        },
        'TypeV': {
            'description': 'Under-determined MQ systems',
            'url_base': 'https://www.mqchallenge.org/challenges/typeV/',
            'sizes': [(50, 25), (55, 28), (60, 30), (65, 33)],
        },
        'TypeVI': {
            'description': 'Rainbow-like MQ systems',
            'url_base': 'https://www.mqchallenge.org/challenges/typeVI/',
            'sizes': [(24, 42), (26, 46), (28, 50), (30, 54)],
        }
    }

    def __init__(self):
        self.results = []
        self.solver = SATSolverQuaternion()

    def generate_test_problem(self, n=10, m=15, seed=42):
        """Generate a random test MQ problem for demonstration"""
        np.random.seed(seed)

        mq = MQProblem()
        mq.n = n
        mq.m = m
        mq.field = 2
        mq.seed = seed

        # Generate random solution
        solution = np.random.randint(0, 2, n)

        # Generate equations that the solution satisfies
        for _ in range(m):
            eq = {
                'quadratic': [],
                'linear': [],
                'constant': 0
            }

            # Random quadratic terms
            num_quad = np.random.randint(1, min(5, n*(n-1)//2 + 1))
            for _ in range(num_quad):
                i = np.random.randint(1, n+1)
                j = np.random.randint(i, n+1)
                if (i, j) not in eq['quadratic']:
                    eq['quadratic'].append((i, j))

            # Random linear terms
            num_lin = np.random.randint(0, min(4, n))
            for _ in range(num_lin):
                i = np.random.randint(1, n+1)
                if i not in eq['linear']:
                    eq['linear'].append(i)

            # Compute value at solution and set constant to make equation satisfied
            value = 0
            for (i, j) in eq['quadratic']:
                value ^= (solution[i-1] & solution[j-1])
            for i in eq['linear']:
                value ^= solution[i-1]

            eq['constant'] = value
            mq.equations.append(eq)

        return mq, solution

    def solve_mq_problem(self, mq_problem, problem_name="Unknown"):
        """Solve a single MQ problem"""
        print(f"\n{'='*60}")
        print(f"  SOLVING: {problem_name}")
        print(f"{'='*60}")

        # Convert MQ to SAT
        converter = MQtoSATConverter(mq_problem)
        clauses, num_vars, var_map, reverse_map = converter.convert()

        # Load into solver
        self.solver = SATSolverQuaternion()
        self.solver.load_clauses(clauses, num_vars)

        # Solve
        print("\n  Solving SAT instance...")
        start_time = time.time()
        result, heur_assign, complexity, exact_assign = self.solver.solve()
        solve_time = time.time() - start_time

        # Extract solution for original variables if SAT
        mq_solution = None
        if result == "SAT" and exact_assign:
            mq_solution = {}
            for i in range(1, mq_problem.n + 1):
                key = ('x', i)
                if key in var_map:
                    sat_var = var_map[key]
                    mq_solution[i] = exact_assign.get(sat_var, False)
                else:
                    mq_solution[i] = False

        # Verify solution
        verified = False
        if mq_solution:
            verified = self._verify_mq_solution(mq_problem, mq_solution)

        # Report results
        sat_count = self.solver.count_satisfied(heur_assign) if heur_assign else 0

        result_data = {
            'name': problem_name,
            'n': mq_problem.n,
            'm': mq_problem.m,
            'result': result,
            'time': solve_time,
            'sat_clauses': num_vars,
            'heuristic_satisfied': sat_count,
            'total_clauses': len(clauses),
            'solution': mq_solution,
            'verified': verified
        }

        self.results.append(result_data)

        print(f"\n  {'='*50}")
        print(f"  RESULT: {result}")
        print(f"  Time: {solve_time:.4f} seconds")
        print(f"  Complexity O(log n): {complexity:.6f}")
        if result == "SAT":
            print(f"  Solution verified: {verified}")
            if mq_solution and len(mq_solution) <= 20:
                sol_str = ''.join(['1' if mq_solution[i] else '0' for i in range(1, len(mq_solution)+1)])
                print(f"  Solution: {sol_str}")
        print(f"  {'='*50}")

        return result_data

    def _verify_mq_solution(self, mq_problem, solution):
        """Verify that solution satisfies all MQ equations"""
        for eq_idx, eq in enumerate(mq_problem.equations):
            value = eq['constant']

            for (i, j) in eq['quadratic']:
                vi = solution.get(i, False)
                vj = solution.get(j, False)
                if vi and vj:
                    value ^= 1

            for i in eq['linear']:
                if solution.get(i, False):
                    value ^= 1

            if value != 0:
                print(f"    Equation {eq_idx} not satisfied!")
                return False

        return True

    def run_demo_challenges(self):
        """Run demonstration challenges with generated problems"""
        print("\n" + "="*70)
        print("  FUKUOKA MQ CHALLENGE SOLVER - QUATERNION DYNAMICS")
        print("  Demo Mode: Testing with generated MQ problems")
        print("="*70)

        demo_sizes = [
            (8, 12, "Tiny"),
            (12, 18, "Small"),
            (16, 24, "Medium"),
            (20, 30, "Large"),
            (24, 36, "XLarge"),
            (28, 42, "XXLarge"),
            (32, 48, "Huge"),
            (36, 54, "Massive"),
        ]

        for n, m, size_name in demo_sizes:
            print(f"\n{'#'*60}")
            print(f"# Testing {size_name}: n={n}, m={m}")
            print(f"{'#'*60}")

            try:
                mq, true_solution = self.generate_test_problem(n, m, seed=n*m)
                result = self.solve_mq_problem(mq, f"Demo_{size_name}_n{n}_m{m}")

                if result['result'] == "SAT":
                    print(f"  ✓ SOLVED in {result['time']:.4f}s")
                else:
                    print(f"  ✗ {result['result']} in {result['time']:.4f}s")

            except Exception as e:
                print(f"  ERROR: {str(e)}")
                import traceback
                traceback.print_exc()

        self._print_summary()

    def run_full_challenge_simulation(self):
        """Simulate the full Fukuoka MQ Challenge progression"""
        print("\n" + "="*70)
        print("  FUKUOKA MQ CHALLENGE - FULL SIMULATION")
        print("  Testing limits of Quaternion Dynamics Solver")
        print("="*70)

        # Progressive difficulty
        difficulties = [
            # (n, m, type_desc)
            (10, 15, "TypeI-Warmup"),
            (12, 18, "TypeI-Easy"),
            (15, 23, "TypeI-Medium"),
            (18, 27, "TypeI-Hard"),
            (20, 30, "TypeI-VeryHard"),
            (22, 33, "TypeI-Expert"),
            (25, 38, "TypeI-Master"),
            (28, 42, "TypeI-Grandmaster"),
            (30, 45, "TypeI-Legendary"),
            (32, 48, "TypeI-Mythic"),
            (35, 53, "TypeI-Impossible"),
            (37, 56, "TypeI-Challenge"),
            (40, 60, "TypeI-Extreme"),
            (45, 68, "TypeI-Ultimate"),
            (50, 75, "TypeI-Beyond"),
        ]

        for n, m, challenge_name in difficulties:
            print(f"\n{'▶'*60}")
            print(f"▶ CHALLENGE: {challenge_name}")
            print(f"▶ Variables: {n}, Equations: {m}")
            print(f"{'▶'*60}")

            try:
                mq, _ = self.generate_test_problem(n, m, seed=n*100+m)

                start = time.time()
                result = self.solve_mq_problem(mq, challenge_name)
                total_time = time.time() - start

                status = "✓ PASSED" if result['result'] == "SAT" and result['verified'] else "✗ FAILED"
                print(f"\n  {status} - Total time: {total_time:.4f}s")

                # Stop if taking too long (for demo purposes)
                if total_time > 60:
                    print("\n  ⚠ Stopping: Time limit reached")
                    print(f"  Maximum solvable size: n={n}, m={m}")
                    break

            except Exception as e:
                print(f"  ERROR: {str(e)}")
                break

        self._print_summary()

    def _print_summary(self):
        """Print summary of all results"""
        print("\n" + "="*70)
        print("  RESULTS SUMMARY")
        print("="*70)
        print(f"\n  {'Problem':<30} {'n':>5} {'m':>5} {'Result':>8} {'Time':>10} {'Verified':>10}")
        print("  " + "-"*70)

        for r in self.results:
            verified_str = "✓" if r.get('verified', False) else "✗"
            print(f"  {r['name']:<30} {r['n']:>5} {r['m']:>5} {r['result']:>8} {r['time']:>10.4f} {verified_str:>10}")

        # Statistics
        total = len(self.results)
        solved = sum(1 for r in self.results if r['result'] == "SAT" and r.get('verified', False))
        total_time = sum(r['time'] for r in self.results)

        print("\n  " + "-"*70)
        print(f"  Total problems: {total}")
        print(f"  Solved & Verified: {solved} ({100*solved/total if total > 0 else 0:.1f}%)")
        print(f"  Total solving time: {total_time:.4f}s")
        print("="*70)


# ============================================================
# FILE UPLOAD HANDLER
# ============================================================
def handle_uploaded_file():
    """Handle user uploaded MQ challenge files"""
    print("\n" + "="*60)
    print("  Upload your MQ Challenge file (.txt, .mq, .gz)")
    print("="*60 + "\n")

    uploaded = files.upload()

    if not uploaded:
        print("[ERROR] No file uploaded")
        return None

    for filename, content in uploaded.items():
        print(f"\nFile: {filename}")
        print(f"Size: {len(content)} bytes")

        # Decompress if needed
        if filename.endswith('.gz'):
            print("Decompressing .gz...")
            content = gzip.decompress(content).decode('utf-8', errors='ignore')
        elif filename.endswith('.xz'):
            print("Decompressing .xz...")
            content = lzma.decompress(content).decode('utf-8', errors='ignore')
        else:
            content = content.decode('utf-8', errors='ignore')

        # Parse MQ problem
        mq = MQProblem()
        if mq.parse_challenge_file(content, filename):
            # Solve
            challenge = FukuokaMQChallenge()
            result = challenge.solve_mq_problem(mq, filename)

            # Save result
            save_mq_result(filename, result, mq)
            return result
        else:
            print("[ERROR] Could not parse MQ problem")

    return None

def save_mq_result(filename, result, mq_problem):
    """Save solution in MQ Challenge format"""
    base = filename
    for ext in ['.gz', '.xz', '.txt', '.mq']:
        if base.endswith(ext):
            base = base[:-len(ext)]

    output_file = f"{base}_solution.txt"

    lines = [
        "# " + "="*50,
        "# MQ Challenge Solution - Quaternion Dynamics Solver",
        "# " + "="*50,
        f"# Problem: {filename}",
        f"# Variables (n): {mq_problem.n}",
        f"# Equations (m): {mq_problem.m}",
        f"# Field: GF({mq_problem.field})",
        f"# Result: {result['result']}",
        f"# Time: {result['time']:.6f} seconds",
        f"# Verified: {result.get('verified', False)}",
        "#"
    ]

    if result['result'] == "SAT" and result.get('solution'):
        lines.append("# Solution (x_1, x_2, ..., x_n):")
        sol = result['solution']
        sol_str = ''.join([str(int(sol[i])) for i in range(1, len(sol)+1)])
        lines.append(f"SOLUTION: {sol_str}")
        lines.append("")
        lines.append("# Individual variables:")
        for i in sorted(sol.keys()):
            lines.append(f"x_{i} = {int(sol[i])}")
    else:
        lines.append("# No solution found (UNSAT)")

    content = '\n'.join(lines)

    with open(output_file, 'w') as f:
        f.write(content)

    print(f"\nSolution saved to: {output_file}")
    files.download(output_file)

    return output_file


# ============================================================
# MAIN EXECUTION
# ============================================================
def main():
    print("="*70)
    print("   FUKUOKA MQ CHALLENGE SOLVER")
    print("   Quaternion Dynamics + PySAT Hybrid Algorithm")
    print("   https://www.mqchallenge.org/")
    print("="*70)

    print("\nSelect mode:")
    print("  1. Run demo challenges (generated problems)")
    print("  2. Run full challenge simulation (progressive difficulty)")
    print("  3. Upload your own MQ Challenge file")
    print("  4. Quick test (small problem)")

    print("\n" + "-"*50)
    mode = input("Enter mode (1/2/3/4) [default=2]: ").strip() or "2"

    if mode == "1":
        challenge = FukuokaMQChallenge()
        challenge.run_demo_challenges()

    elif mode == "2":
        challenge = FukuokaMQChallenge()
        challenge.run_full_challenge_simulation()

    elif mode == "3":
        handle_uploaded_file()

    elif mode == "4":
        print("\n  Running quick test...")
        challenge = FukuokaMQChallenge()
        mq, solution = challenge.generate_test_problem(8, 12, seed=42)
        result = challenge.solve_mq_problem(mq, "QuickTest_n8_m12")
        challenge._print_summary()

    else:
        print("Invalid mode. Running demo...")
        challenge = FukuokaMQChallenge()
        challenge.run_demo_challenges()

# Run the solver
if __name__ == "__main__":
    main()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 21.7 MB/s eta 0:00:00
   FUKUOKA MQ CHALLENGE SOLVER
   Quaternion Dynamics + PySAT Hybrid Algorithm
   https://www.mqchallenge.org/

Select mode:
  1. Run demo challenges (generated problems)
  2. Run full challenge simulation (progressive difficulty)
  3. Upload your own MQ Challenge file
  4. Quick test (small problem)

--------------------------------------------------
Enter mode (1/2/3/4) [default=2]: 2

  FUKUOKA MQ CHALLENGE - FULL SIMULATION
  Testing limits of Quaternion Dynamics Solver

▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶
▶ CHALLENGE: TypeI-Warmup
▶ Variables: 10, Equations: 15
▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶

  SOLVING: TypeI-Warmup

  Converting MQ to SAT...
    SAT Variables: 86
    SAT Clauses: 314

  Solving SAT instance...
  [1] Quaternionic approximation O(log n)...
      Heuristic: 246/314 (78.34%)
  [2] Exact decision with PySAT (CDCL)...

  RESULT: SAT
  Time